In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers

tensorflow.keras.utils.set_random_seed(42)

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/nvidia/nvidia_stock_prices.csv')
df = pd.DataFrame(data)

df['Date'] = pd.to_datetime(df['Date'])
df['year'] = df['Date'].dt.year
df['month'] = df['Date'].dt.month
df['day'] = df['Date'].dt.day
df['dayofweek_num'] = df['Date'].dt.dayofweek

In [ ]:
correlation_matrix = df[['year', 'month', 'day', 'High']].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.show()

In [ ]:
df.set_index('Date', inplace=True)

plt.figure(figsize = (8,6))
plt.title('Nvidia Stock Prices')
plt.plot(df['High'])
plt.show()

In [ ]:
data = df[['Close']].copy()
print(data.columns)

scaler = StandardScaler()
scaled_data = scaler.fit_transform(data)

def create_sequences(data, seq_length):
    x, y = [], []
    for i in range(seq_length, len(data)):
        x.append(data[i - seq_length:i, 0])
        y.append(data[i, 0])
    return np.array(x), np.array(y)

seq_length = 60
X, y = create_sequences(scaled_data, seq_length)

print(X.shape)

# Reshape to format [samples, time steps, features]
X = X.reshape((X.shape[0], X.shape[1], 1))

In [ ]:
X.shape

In [ ]:
model = Sequential()
model.add(Bidirectional(LSTM(100, return_sequences=True), input_shape=(X.shape[1], 1), merge_mode='sum'))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(100, return_sequences=False), input_shape=(X.shape[1], 1), merge_mode='sum'))
model.add(Dropout(0.3))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mae')

model.summary()

In [ ]:
callback = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

split = int(0.8 * X.shape[0])
X_train, y_train = X[:split], y[:split]
X_test, y_test = X[split:], y[split:]

history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test), callbacks=[callback])

In [ ]:
predicted = model.predict(X_test)
predicted = scaler.inverse_transform(predicted.reshape(-1, 1))
actual = scaler.inverse_transform(y_test.reshape(-1, 1))

plt.figure(figsize=(12, 6))
plt.plot(actual, label='True price')
plt.plot(predicted, label='LSTM prediction')
plt.title('NVIDIA Stock Price Forecast')
plt.xlabel('Days')
plt.ylabel('Price')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
future_days = 60
last_sequence = scaled_data[-seq_length:].reshape(1, seq_length, 1)

future_predictions = []

for _ in range(future_days):
    next_pred = model.predict(last_sequence)[0][0]
    future_predictions.append(next_pred)
    last_sequence = np.append(last_sequence[:, 1:, :], [[[next_pred]]], axis=1)

future_predictions = scaler.inverse_transform(np.array(future_predictions).reshape(-1, 1))

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(range(len(actual)), actual, label='True price (test)')
plt.plot(range(len(actual), len(actual) + future_days), future_predictions, label='Prediction for 3 months', linestyle='--')
plt.title('NVIDIA Stock Price Forecast (Autoregressive LSTM)')
plt.xlabel('Days')
plt.ylabel('Price')
plt.legend()
plt.grid(True)
plt.show()